- https://github.com/JaidedAI/EasyOCR
- https://www.jaided.ai/easyocr/

In [3]:
import os 
from pathlib import Path

os.chdir(Path.cwd().parent)
print(f"Current working directory: {os.getcwd()}")

Current working directory: /Users/josingh/hobby/plainfigures/app


### Using local OCR Model

In [5]:
import easyocr
reader = easyocr.Reader(['en']) # this needs to run only once to load the model into memory
result = reader.readtext('data/sg-invoice.jpg')

/Users/josingh/hobby/plainfigures/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [6]:
result

[([[np.int32(276), np.int32(214)],
   [np.int32(486), np.int32(214)],
   [np.int32(486), np.int32(286)],
   [np.int32(276), np.int32(286)]],
  'INV24.SG',
  np.float64(0.9593737334586523)),
 ([[np.int32(281), np.int32(279)],
   [np.int32(493), np.int32(279)],
   [np.int32(493), np.int32(303)],
   [np.int32(281), np.int32(303)]],
  'IN OICE SOFTWARE',
  np.float64(0.48042818378501245)),
 ([[np.int32(158), np.int32(446)],
   [np.int32(250), np.int32(446)],
   [np.int32(250), np.int32(478)],
   [np.int32(158), np.int32(478)]],
  'Bill To:',
  np.float64(0.9909974070674015)),
 ([[np.int32(157), np.int32(477)],
   [np.int32(547), np.int32(477)],
   [np.int32(547), np.int32(515)],
   [np.int32(157), np.int32(515)]],
  '[Customer Business Name]',
  np.float64(0.8878570459492514)),
 ([[np.int32(1116), np.int32(480)],
   [np.int32(1438), np.int32(480)],
   [np.int32(1438), np.int32(512)],
   [np.int32(1116), np.int32(512)]],
  'Invoice Date: 01/06/2021',
  np.float64(0.5654567107922048)),
 ([[n

### Using GPT

In [ ]:
import base64
from openai import OpenAI
import os

# Initialize the OpenAI client with your API key
# Ensure you have your OPENAI_API_KEY set as an environment variable or replace "YOUR_OPENAI_API_KEY"
client = OpenAI() 

# Function to encode the image to base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# Path to your image file
image_path = "data/sg-invoice.jpg"

# Encode the image
base64_image = encode_image(image_path)

# Prepare the message for the API call
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Extract all text from this image in a structured JSON format."},
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_image}"
                },
            },
        ],
    }
]

# Make the API call
try:
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # Specify the GPT-4o mini model
        messages=messages,
        max_tokens=1000,  # Adjust as needed for the expected text length
    )
    extracted_text = response.choices[0].message.content
    print("Extracted Text:")
    print(extracted_text)

except Exception as e:
    print(f"An error occurred: {e}")

Extracted Text:
```json
{
  "invoice": {
    "billTo": {
      "customerBusinessName": "[Customer Business Name]",
      "customerBusinessAddress": "[Customer Business Address]",
      "customerGSTReg": "[Customer GST Reg Nr]"
    },
    "invoiceDate": "01/06/2021",
    "dueDate": "15/06/2021",
    "invoiceNumber": "1291"
  },
  "items": [
    {
      "description": "products",
      "qty": 5,
      "unitPrice": 594.00,
      "subtotal": 2970.00
    },
    {
      "description": "services",
      "qty": 1,
      "unitPrice": 1900.00,
      "subtotal": 1900.00
    }
  ],
  "subtotal": 4870.00,
  "gst": {
    "rate": "7%",
    "amount": 340.90
  },
  "total": 5210.90,
  "supplier": {
    "supplierBusinessName": "[Supplier Business Name]",
    "supplierBusinessAddress": "[Supplier Business Address]",
    "supplierGSTReg": "[GST Reg Nr]",
    "bankDetails": {
      "bankName": "[Bank name]",
      "accountNumber": "[Bank account number]",
      "swiftBic": "[SWIFT / BIC]",
      "otherPaym